In [61]:
import os
import glob
import csv

import pandas as pd
from sklearn.model_selection import KFold, StratifiedKFold

from keras.applications import mobilenet
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras import layers, models
import numpy as np
import tensorflow as tf

from csv import reader

# libraries for outputting camera images
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from IPython.display import clear_output

import pickle

import pylab as pl

%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [58]:
img_size=224
n_inst = 424
n_cats = 6
dir_name = f"finalModel_n{n_inst}_{n_cats}cats"
columns = [str(i) for i in range(n_cats)]

# 'data/train_cv_224/training_labels_handOverFace_n424.csv'
if n_inst == 871:
    csv_filename = f'data/train_cv_224/handOverFace_n871_{n_cats}cats_1.csv'
else:
    csv_filename = f'data/train_cv_224/training_labels_handOverFace_n424_{n_cats}cats.csv'
targetDir = 'data/train_cv_224'

train_data = pd.read_csv(csv_filename, sep=',', header=0)

# settuping the kfolds
kf = KFold(n_splits=10, shuffle=True)
skf = StratifiedKFold(n_splits=5, shuffle=True)



idg = ImageDataGenerator(preprocessing_function=mobilenet.preprocess_input)
idg_train = ImageDataGenerator(
    preprocessing_function=mobilenet.preprocess_input, width_shift_range=[-25, 25], 
    height_shift_range=[-25, 25], horizontal_flip=True, brightness_range=[0.6,1.3],
    channel_shift_range=15
)

In [59]:
def create_new_model():
    base_model = mobilenet.MobileNet(weights='imagenet', 
                                     include_top=False, 
                                     input_shape=(img_size, img_size, 3))
    for layer in base_model.layers:
        layer.trainable = False
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(len(columns), activation='softmax')(x)
    model = models.Model(inputs=base_model.input, outputs=x)
    
    model.compile(optimizer='adam', 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

early_stopping_monitor = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0,
    patience=0,
    verbose=0,
    mode='auto',
    baseline=None,
    restore_best_weights=True
)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=dir_name+'/mobileNet_n'+str(n_inst)+'_'+str(n_cats)+'cats-{epoch:02d}-{val_loss:.2f}.h5', 
    verbose=1, monitor='val_accuracy', save_best_only=False
)

In [60]:
%%notify
### FOR GETTING A GOOD MODEL
### FOR CROSS VALIDATION
## running only one of it
valid_accuracy = []
valid_loss = []
batch_size=32
num_epochs=20
num_samples = train_data.shape[0]

for train_index, val_index in list(kf.split(np.zeros(num_samples), train_data)):
    training_data = train_data.iloc[train_index]
    validation_data = train_data.iloc[val_index]
    
    # one hot encoding because it is softmax    
    train_data_generator = idg.flow_from_dataframe(dataframe=training_data, directory=targetDir,
                                                  x_col="filename", 
                                                   y_col=columns,
                                                  shuffle=True,
                                                  target_size=(img_size, img_size), 
                                                   batch_size=batch_size, 
                                                  class_mode='raw')
    valid_data_generator = idg.flow_from_dataframe(dataframe=validation_data, directory=targetDir,
                                                  x_col="filename", 
                                                   y_col=columns,
                                                  shuffle=True,
                                                  target_size=(img_size, img_size), 
                                                   batch_size=batch_size,
                                                  class_mode='raw')
    
    model = create_new_model()
    
    step_size = train_data_generator.n//train_data_generator.batch_size
    step_size_valid = valid_data_generator.n//valid_data_generator.batch_size

    with tf.device('/device:GPU:0'):
        history = model.fit(train_data_generator, epochs=num_epochs, steps_per_epoch=step_size, 
                        validation_data=valid_data_generator,
                       validation_steps=step_size_valid,
                        callbacks=[model_checkpoint_callback])
    
    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names, results))
    
    valid_accuracy.append(results['accuracy'])
    valid_loss.append(results['loss'])
    
    output_dir_name = os.path.join(dir_name, f"history.pickle")
    with open(output_dir_name, "wb") as f:
        pickle.dump(history.history, f)
    break

Found 2289 validated image filenames.
Found 255 validated image filenames.
Epoch 1/20
71/71 [==============================] - 34s 424ms/step - loss: 0.8283 - accuracy: 0.7108 - val_loss: 1.0786 - val_accuracy: 0.6116

Epoch 00001: saving model to finalModel_n424_6cats/mobileNet_n424_6cats-01-1.08.h5
Epoch 2/20
71/71 [==============================] - 28s 394ms/step - loss: 0.1856 - accuracy: 0.9322 - val_loss: 0.8021 - val_accuracy: 0.7366

Epoch 00002: saving model to finalModel_n424_6cats/mobileNet_n424_6cats-02-0.80.h5
Epoch 3/20
71/71 [==============================] - 28s 402ms/step - loss: 0.1431 - accuracy: 0.9480 - val_loss: 0.2660 - val_accuracy: 0.9062

Epoch 00003: saving model to finalModel_n424_6cats/mobileNet_n424_6cats-03-0.27.h5
Epoch 4/20
71/71 [==============================] - 30s 416ms/step - loss: 0.0979 - accuracy: 0.9629 - val_loss: 0.2893 - val_accuracy: 0.9196

Epoch 00004: saving model to finalModel_n424_6cats/mobileNet_n424_6cats-04-0.29.h5
Epoch 5/20
71/71 

<IPython.core.display.Javascript object>

In [44]:
output_dir_name = os.path.join(dir_name, f"history.pickle")
with open(output_dir_name, "wb") as f:
    pickle.dump(history.history, f)